In [5]:
import numpy as np
import LoadData as ld
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import mne
from time import perf_counter
import keras
import tensorflow as tf


def createChannelInfo(no_channels):
    '''This function creates an info object 
    that is required later to convert the eeg data from np data
    to mne.io.raw data.

    no_channels: number of channels used in the eeg data

    returns info object
    '''
    ch_names = []
    for i in range(no_channels):
        ch_names.append('Channel' +"-"+ str(i))   #for this code I have just created dummy channel names. We can have actual channel names
    info = mne.create_info(ch_names = ch_names, sfreq = 250, ch_types='eeg') 
    return info 

def bandPassFilterReshape(raw_data, info):
    '''
    First converts the np data into mne.io.raw data and
    uses createEpoch funcn to convert the data into epochs
    raw_data : eeg data
    info: info object we created before which has information about the channel
    
    returns epoch data which will be applied for CSP
    '''
    raw_data = np.transpose(raw_data)
    features = mne.io.RawArray(raw_data, info)
    features.filter(7.0, 40.0)
    features_data = features.get_data()
    features_data = np.transpose(features_data)
    # features_data = createEpoch(features_data, 250, 64)
    return features_data

In [6]:
raw_data_hand = ld.loadData()   
raw_data_feet = ld.loadData()    #loading eeg data of any two movements of ur choosing

test_data_hand = ld.loadData()
test_data_feet = ld.loadData() 

info = createChannelInfo(22)     #we will need the info object to convert the eeg data from np data to raw_eeg data which we will need for bandpass filtering

features_hand = bandPassFilterReshape(raw_data_hand, info)   #this converts the eeg data into epochs, which will be needed for CSP
features_feet = bandPassFilterReshape(raw_data_feet, info)

features_test_hand = bandPassFilterReshape(test_data_hand, info)
features_test_feet = bandPassFilterReshape(test_data_feet, info)

The file name provided is correct. Thank you!
The file name provided is correct. Thank you!
The file name provided is correct. Thank you!
The file name provided is correct. Thank you!
Creating RawArray with float64 data, n_channels=22, n_times=51750
    Range : 0 ... 51749 =      0.000 ...   206.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 413 samples (1.652 s)

Creating RawArray with float64 data, n_channels=22, n_times=51750
    Range : 0 ... 51749 =      0.000 ...   206.996 secs
Read

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Creating RawArray with float64 data, n_channels=22, n_times=53250
    Range : 0 ... 53249 =      0.000 ...   212.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 413 samples (1.652 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


In [3]:
features = np.concatenate((features_hand, features_feet))

features_test = np.concatenate((features_test_hand, features_test_feet))
label_test = np.concatenate((np.zeros(features_test_hand.shape[0]), np.ones(features_test_feet.shape[0])))

dataset_test = tf.data.Dataset.from_tensor_slices((features_test, label_test))
dataset_test = dataset_test.shuffle(buffer_size = features_test.shape[0]).batch(32)


label = np.concatenate((np.zeros(features_hand.shape[0]), np.ones(features_feet.shape[0])))

dataset = tf.data.Dataset.from_tensor_slices((features, label))

dataset = dataset.shuffle(buffer_size= features.shape[0]).batch(32)

# print(dataset_test)

In [7]:
input = keras.Input(shape=(features.shape[1],))

hl1 =  keras.layers.Dense(128, activation= keras.layers.LeakyReLU(0.3))(input)

hl2 = keras.layers.Dense(128, activation= keras.layers.LeakyReLU(0.3))(hl1)

hl3 = keras.layers.Dense(128, activation = keras.layers.LeakyReLU(0.3)) (hl2)

output = keras.layers.Dense(2, activation = "softmax")(hl3)


model =  keras.Model(input, output)

print(model.summary())
                                                     


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 22)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │         2,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,226 (141.51 KB)

 Trainable params: 36,226 (141.51 KB)

 Non-trainable params: 0 (0.00 B)

None


In [8]:
model.compile(optimizer="adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

model.fit(dataset, epochs = 50, validation_data = dataset_test)


Epoch 1/50
3235/3235 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.5171 - loss: 0.7150 - val_accuracy: 0.5574 - val_loss: 0.6778
Epoch 2/50
3235/3235 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5366 - loss: 0.6875 - val_accuracy: 0.5610 - val_loss: 0.6767
Epoch 3/50
3235/3235 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5362 - loss: 0.6867 - val_accuracy: 0.5609 - val_loss: 0.6752
Epoch 4/50
3235/3235 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5431 - loss: 0.6856 - val_accuracy: 0.5604 - val_loss: 0.6784
Epoch 5/50
3235/3235 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.5442 - loss: 0.6850 - val_accuracy: 0.5807 - val_loss: 0.6711
Epoch 6/50
3235/3235 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5471 - loss: 0.6827 - val_accuracy: 0.5686 - val_loss: 0.6759
Epoch 7/50
3235/3235 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.5503 - loss: 0.6828 - val_accuracy: 0.5680 - val_loss: 0.6755
Epoch 8/50
3235/3235 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5540 - loss: 0.6811 -